<a href="https://colab.research.google.com/github/NbtKmy/gc_workshops/blob/main/Embedding_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embedding


[Offizielles Dokument](https://platform.openai.com/docs/guides/embeddings/use-cases)


[OpenAI Cookbook](https://github.com/openai/openai-cookbook/tree/main/examples)

Beispiel https://de.wikipedia.org/wiki/Ukrainisches_Kulturerbe_w%C3%A4hrend_der_russischen_Invasion_2022

In [12]:
!pip install -q openai pandas

In [13]:
import pandas as pd
from google.colab import files

uploaded = files.upload()
input_csv = next(iter(uploaded))

df = pd.read_csv(input_csv, header=None, names=["text"])
# display(df)

Saving embedding_txts.csv to embedding_txts.csv


In [14]:
import openai


my_api_key = ''#@param{type:'string'} 
openai.api_key = my_api_key

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

df['ada_embedding'] = df.text.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
df.to_csv('embedded_source.csv', index=False)


In [15]:
from openai.embeddings_utils import cosine_similarity

def search_sim_texts(df, question_text, n=3, pprint=True):
   print(question_text)
   question_text = question_text.replace("\n", " ")
   model = "text-embedding-ada-002"
   embedding = openai.Embedding.create(input = [question_text], model=model)['data'][0]['embedding']
   df['similarities'] = df.ada_embedding.apply(lambda x: cosine_similarity(x, embedding))
   res = df.sort_values("similarities", ascending=False).head(n)
   return res


res = search_sim_texts(df, "Wo ist das wichtige Kulturerbe in Ukraine zu finden?", n=3)
display(res)


Wo ist das wichtige Kulturerbe in Ukraine zu finden?


,text,ada_embedding,similarities
1,In der Ukraine gibt es sieben Welterbestätten ...,"[0.027888653799891472, 0.0019267756724730134, ...",0.885547
2,"Ende April 2022 wurde berichtet, dass die Ukra...","[0.0038406106177717447, -0.021663345396518707,...",0.865676
0,Das ukrainische Kulturerbe erlitt während der ...,"[0.0005776059115305543, -0.006201579701155424,...",0.861394


## ChatBot mit Embedding

In [6]:
!pip install -q openai pandas gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 6.5 MB/s eta 0:

In [2]:
# Zuerst Instruktion eingeben
from google.colab import files

uploaded = files.upload()




Saving embedded_source.csv to embedded_source.csv


In [12]:
import openai
from openai.embeddings_utils import cosine_similarity
import pandas as pd
import numpy as np
import gradio as gr


def search_sim_texts(df, question_text, n=3, pprint=True):
   # print(question_text)
   # display(df)
   question_text = question_text.replace("\n", " ")
   model = "text-embedding-ada-002"
   embedding = openai.Embedding.create(input = [question_text], model=model)['data'][0]['embedding']
   df['similarities'] = df.ada_embedding.apply(lambda x: cosine_similarity(x, embedding))
   res = df.sort_values("similarities", ascending=False).head(n)
   return res


def chat(question_text):
    
    my_api_key = ''#@param{type:'string'}
    openai.api_key = my_api_key

    filename = 'embedded_source.csv'#@param{type:'string'}
    df = pd.read_csv(filename, header=0)
    df['ada_embedding'] = df.ada_embedding.apply(eval).apply(np.array)
    
   

    messages = []
    
    system_msg = 'Du spielst als ChatBot und gibst möglichst genaue Antwort.'   
    messages.append({'role': 'system', 'content': system_msg})

   
    res = search_sim_texts(df, question_text, n=3, pprint=True)
    res_list = res["text"].tolist()

    question_with_few_Shots = 'Bitte Antworten Sie auf die Frage mit Hilfe der folgenden Information:\n' \
                              + '\n\n'.join(res_list) \
                              + '\n---\n' \
                              + 'Frage:\n' + question_text

    
    messages.append ({'role': 'user', 'content': question_with_few_Shots})
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=messages,
        temperature=0.5,
    )
    reply = response['choices'][0]['message']['content']
    

    reply_message = '🤖 Antwort: ' + reply
    
    return reply_message


if __name__ == '__main__':

    
    gr.Interface(
        fn=chat, 
        inputs=[
            gr.Textbox( lines=2, placeholder='Hier die Fragen eingeben', label="Frage", ),
        ],
        outputs=[
            gr.Textbox(lines=7, label="Output",),
        ],
    ).launch()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>